# Genetic Algorithm Implementation in Python
Genetic Algorithm Implementation in Python — By Ahmed F. Gad<br>
LinkedIn: https://www.linkedin.com/pulse/genetic-algorithm-implementation-python-ahmed-gad/<br>
AI 研習社：https://ai.yanxishe.com/page/TextTranslation/1207

Source code : https://github.com/ahmedfgad/GeneticAlgorithmPython <br>
My clone: https://github.com/hcchengithub/GeneticAlgorithmPython


In [1]:
import peforth  # v1.24

reDef unknown
reDef -->


## Tutorial Example
The tutorial starts by presenting the equation that we are going to implement. The equation is shown below:

Y = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + w6x6

The equation has 6 inputs (x1 to x6) and 6 weights (w1 to w6) as shown and inputs values are (x1,x2,x3,x4,x5,x6)=(4,-2,7,5,11,1). We are looking to find the parameters (weights) that maximize such equation. The idea of maximizing such equation seems simple. The positive input is to be multiplied by the largest possible positive number and the negative number is to be multiplied by the smallest possible negative number. But the idea we are looking to implement is how to make GA do that its own in order to know that it is better to use positive weight with positive inputs and negative weights with negative inputs. Let us start implementing GA.

At first, let us create a list of the 6 inputs and a variable to hold the number of weights as follows:

In [2]:
equation_inputs = [4,-2,3.5,5,-11,-4.7]
num_weights = len(equation_inputs)

In [3]:
%f equation_inputs -->
%f num_weights -->

equation_inputs --> [4, -2, 3.5, 5, -11, -4.7] (<class 'list'>)
num_weights --> 6 (<class 'int'>)


The next step is to define the initial population. Based on the number of weights, each chromosome (solution or individual) in the population will definitely have 6 genes, one gene for each weight. But the question is how many solutions per the population? There is no fixed value for that and we can select the value that fits well with our problem. But we could leave it generic so that it can be changed in the code. Next, we create a variable that holds the number of solutions per population, another to hold the size of the population, and finally, a variable that holds the actual initial population:

In [4]:
import numpy

sol_per_pop = 8  # solutions per population 

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

After importing the numpy library, we are able to create the initial population randomly using the numpy.random.uniform function. According to the selected parameters, it will be of shape (8, 6). That is 8 chromosomes and each one has 6 genes, one for each weight. After running this code, the population is as follows:

#### 實驗
把 sol_per_pop 改成 80 (population 也就是 chromosome 染色體的條數 也就是一代中缸子裡有多少可能性參與競爭) 結果由原來的 ~40 分上升到 ~130 分。

In [5]:
%f new_population . cr

[[-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -0.9724159 ]
 [-1.60648454  2.98731    -0.17179483  2.13952118  0.83779334  0.7196666 ]
 [-1.74953969 -3.56136451 -0.22888881 -0.28895321  1.16381269 -1.10860643]
 [ 1.77718335 -3.93334683 -0.45756823  2.93656102  2.50230994  3.74874976]
 [ 0.66986951 -3.03126141 -3.63986569 -0.15803427  3.93955901  2.03904557]
 [ 0.45886834 -2.44420888  2.94516232 -1.74789717 -2.94093216  3.55181148]
 [ 2.01301423  3.23521924  2.723441   -3.98261914  3.44337194  1.26286695]
 [ 0.10465165  2.72794357 -2.11283096  0.06572773 -2.74471276 -1.37170897]]


Note that it is generated randomly and thus it will definitely change when get run again.

After preparing the population, next is to follow the flowchart in figure 1. Based on the fitness function, we are going to select the best individuals within the current population as parents for mating (挑選最佳個體作為交配的親代). Next is to apply the GA variants (crossover and mutation) 基因洗牌 與 突變 to produce the offspring of the next generation, creating the new population by appending both parents and offspring, and repeating such steps for a number of iterations/generations. The next code applies these steps:

In [6]:
# import GA  
# 分開放，方便重新定義埋進 breakpoints 研究學習
from GA import *

In [7]:
num_generations = 5
num_parents_mating = 4
for generation in range(num_generations):
    # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(equation_inputs, new_population)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(pop_size[0]-parents.shape[0], num_weights))
                                                            # 子代個數，不懂為何要這樣算？ 直接 hard coded 不就好了？

    # Adding some variations to the offsrping using mutation.    _debug_
    offspring_mutation = mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

In [8]:
%f pop_size -->
%f parents :> shape -->
%f ( pop_size[0]-parents.shape[0] ) pop_size :> [0] parents :> shape[0] - -->  # 子代個數，不懂為何要這樣算？

pop_size --> (8, 6) (<class 'tuple'>)
parents :> shape --> (4, 6) (<class 'tuple'>)
( pop_size[0]-parents.shape[0] ) pop_size :> [0] parents :> shape[0] - --> 4 (<class 'int'>)
子代個數，不懂為何要這樣算？



In [9]:
%f new_population . cr

[[-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -2.50548442]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.82607565]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.64899546]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -2.02495388]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.36188952]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -3.03634722]
 [-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -1.19258244]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -1.55320021]]


In [12]:
%f fitness ==> # 給 population 評分
%f parents ==> # 從 population 當中挑出最好的種子
%f offspring_crossover ==> # 種子基因洗牌
%f offspring_mutation ==>  # 種子基因突變

fitness ==>
 [-1.00000000e+11  3.95704757e+01  3.93110735e+01  3.91365103e+01
 -1.00000000e+11  3.98550832e+01 -1.00000000e+11 -1.00000000e+11] (<class 'numpy.ndarray'>)
給 population 評分

parents ==>
 [[-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -2.50548442]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.82607565]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.64899546]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -2.02495388]] (<class 'numpy.ndarray'>)
從 population 當中挑出最好的種子

offspring_crossover ==>
 [[-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.36188952]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -3.03634722]
 [-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -1.19258244]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -1.55320021]] (<class 'numpy.ndarray'>)
種子基因洗牌

offspring_mutation ==>
 [[-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.

## 這樣就已經完成了，靠！出乎意外的簡單。
上面這 8 對 chromosome 所組成的 population 是進化到目前的最終結果。以下從這群 population 當中列出結果 

In [13]:
# Getting the best solution after iterating finishing all generations.
# At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(equation_inputs, new_population)

# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])


Best solution :  [[[-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276
   -3.03634722]]]
Best solution fitness :  [44.11477039]


In [15]:
%f fitness ==> # 給 population 評分
%f best_match_idx --> # 挑出最好的


fitness ==>
 [43.22156528 43.12649401 42.29421709 40.48962302 40.94481918 44.11477039
 37.05092593 38.27238075] (<class 'numpy.ndarray'>)
給 population 評分

best_match_idx --> (array([5], dtype=int64),) (<class 'tuple'>)
挑出最好的



## 以下一一深入說明

<font size="5">T</font>he current number of generations is 5. It is selected to be small for presenting results of all generations within the tutorial. There is a module named GA that holds the implementation of the algorithm.

The first step is to find the fitness value of each solution within the population using the GA.cal_pop_fitness function. The implementation of such function inside the GA module is as follows:

In [16]:
def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calculates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

In [25]:
%f new_population ==> 
%f equation_inputs ==>
%f new_population equation_inputs * ==> # 對 population 裡的每一組 Chromosome 裡的 gene 抓對乘上 equation inputs
%f -1.74953969 4 * --> -3.56136451 -2 * -->

new_population ==>
 [[-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -2.50548442]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.82607565]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.64899546]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -2.02495388]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.36188952]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -3.03634722]
 [-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -1.19258244]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -1.55320021]] (<class 'numpy.ndarray'>)
equation_inputs ==>
 [4, -2, 3.5, 5, -11, -4.7] (<class 'list'>)
new_population equation_inputs * ==>
 [[-6.99815875  7.12272902 -0.80111085 -2.56469977 34.68702884 11.77577679]
 [-6.99815875  7.12272902 -0.80111085  0.32863867 30.19184036 13.28255557]
 [-6.99815875  7.12272902 -0.80111085  0.32863867 30.19184036 12.45027866]
 [-1.38443029  0.97203636

上面這個 cal_pop_fitness() 很簡單，照定義把 population 裡面每個 chromosome 的評分算出來。但是隨應用要修改的就是它。其中 equation_inputs 將來可能就是個 simulator 了; 而 pop (population) 一開始是亂猜的，隨後將「進化」出好的結果。

<font size="5">T</font>he fitness function accepts both the equation inputs values (x1 to x6) in addition to the population. The fitness value is calculated as the sum of product  (SOP 就是文章一開始那方程式的 Y 值) between each input and its corresponding gene (weight) according to our function. According to the number of solutions per population, there will be a number of SOPs. As we previously set the number of solutions to 8 in the variable named sol_per_pop, there will be 8 SOPs as shown below:

In [26]:
SOPs = cal_pop_fitness(equation_inputs,new_population)
%f SOPs . cr

[43.22156528 43.12649401 42.29421709 40.48962302 40.94481918 44.11477039
 37.05092593 38.27238075]


<font size="5">N</font>ote that the higher the fitness value the better the solution.

After calculating the fitness values for all solutions, next is to select the best of them as parents in the mating pool according to the next function GA.select_mating_pool. Such function accepts the population, the fitness values, and the number of parents needed. It returns the parents selected. Its implementation inside the GA module is as follows:

In [ ]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))  # 產生 4X6 的 matrix 裡面都是 float 亂數，我覺得這行有機會省略 <== 錯
    for parent_num in range(num_parents): # 轉 n 圈找出最大的 n 個 
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))  # np.where() 的結果是個 tuple
        max_fitness_idx = max_fitness_idx[0][0]  # tuple 裡面是個 array 再裡面才是 index 值，依它就對了。
        # parents[parent_num, :] = pop[max_fitness_idx, :]  # 我覺得用 parents.append(pop[max_fitness_idx]) 即可 <== 錯
        parents[parent_num] = pop[max_fitness_idx]  # 可以這樣簡化，原式的 ", : " 冗長確實沒有必要。
        fitness[max_fitness_idx] = -99999999999  # 每轉一圈找出最大值，槓掉已經找到的
    return parents

In [ ]:
# 這段改寫只是好玩，結果沒有原來的好。

debug = False
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    # parents = numpy.empty((num_parents, pop.shape[1]))  # 產生 4X6 的 matrix 裡面都是 float 亂數，我覺得這行有機會省略
                                                          # 我以為這行可省略，錯！ 還是要有個初始 shape 才行！
    parents = numpy.empty((0, pop.shape[1]))  # 不能省略，但是可以簡化到像這樣空無一物
    if debug : peforth.push(locals()).ok("bp11>",cmd='to _locals_')        
    for parent_num in range(num_parents): # 轉 n 圈找出最大的 n 個 
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))  # np.where() 的結果是個 tuple
        max_fitness_idx = max_fitness_idx[0][0]  # tuple 裡面是個 array 再裡面才是 index 值，依它就對了。
        # parents[parent_num, :] = pop[max_fitness_idx, :]  # 我覺得用 parents.append(pop[max_fitness_idx]) 即可 <==錯!無此物且 np.append 也不合用
        parents = numpy.vstack((parents, pop[max_fitness_idx]))  # 這樣改寫，沒有比原來的好 :-( 
        fitness[max_fitness_idx] = -99999999999  # 每轉一圈找出最大值，已經找到過的就槓掉
        if debug : peforth.push(locals()).ok("bp22>",cmd='to _locals_')        
    return parents

<font size="5">B</font>ased on the number of parents required as defined in the variable num_parents_mating, the function creates an empty array to hold them as in this line:

    parents = numpy.empty((num_parents, pop.shape[1]))

Looping through the current population, the function gets the index of the highest fitness value because it is the best solution to be selected according to this line:

    max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
    
This index is used to retrieve the solution that corresponds to such fitness value using this line:

    parents[parent_num, :] = pop[max_fitness_idx, :]
    parents[parent_num] = pop[max_fitness_idx]  # 這樣即可，上式過度冗贅。
    
To avoid selecting such solution again, its fitness value is set to a very small value that is likely to not be selected again which is -99999999999. The parents array is returned finally which will be as follows according to our example:    

In [27]:
parents_array = select_mating_pool(new_population, SOPs, num_parents_mating)
%f parents_array . cr

[[-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -3.03634722]
 [-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -2.50548442]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.82607565]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.64899546]]


Note that these three (四個吧？) parents are the best individuals within the current population based on their fitness values which are 18.24112489, 17.0688537, 15.99527402, and 14.40299221, respectively. (這些值都是亂數，每次都不一樣)

Next step is to use such selected parents for mating in order to generate the offspring. The mating starts with the crossover operation according to the GA.crossover function. This function accepts the parents and the offspring size. It uses the offspring size to know the number of offspring to produce from such parents. Such a function is implemented as follows inside the GA module:

In [ ]:
debug = False 
def crossover(parents, offspring_size):  # offspring_size --> (4, 6) (<class 'tuple'>)
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)  # 整條 chromosome 的中央, 3, 何必每次重算？ 

    for k in range(offspring_size[0]):
        # Index of the first parent to mate. --> 0 1 2 3
        parent1_idx = k%parents.shape[0]  # parents :> shape[0] --> 4 
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0] --> 1 2 3 0
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]  # 染色體的前半段 來自母親
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:] # 染色體的後半段 來自父親 
        if debug : peforth.push(locals()).ok("bp33>",cmd='to _locals_')     
    return offspring  # 子代的染色體一半來自母親、另一半來自父親 

<font size="5">T</font>he function starts by creating an empty array based on the offspring size as in this line:

    offspring = numpy.empty(offspring_size)  # offspring_size --> (4, 6) (<class 'tuple'>)

Because we are using single point crossover, we need to specify the point at which crossover takes place. The point is selected to divide the solution into two equal halves according to this line:

    crossover_point = numpy.uint8(offspring_size[1]/2)

Then we need to select the two parents to crossover. The indices of these parents are selected according to these two lines:

    parent1_idx = k%parents.shape[0]
    parent2_idx = (k+1)%parents.shape[0]

The parents are selected in a way similar to a ring. The first with indices 0 and 1 are selected at first to produce two offspring. If there still remaining offspring to produce, then we select the parent 1 with parent 2 to produce another two offspring. If we are in need of more offspring, then we select the next two parents with indices 2 and 3. By index 3, we reached the last parent. If we need to produce more offspring, then we select parent with index 3 and go back to the parent with index 0, and so on.

The solutions after applying the crossover operation to the parents are stored into the offspring variable and they are as follows:

In [34]:
offsprings = crossover(parents, (4,6))
%f offsprings ==>

offsprings ==>
 [[-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.82607565]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.64899546]
 [-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -2.02495388]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -2.50548442]] (<class 'numpy.ndarray'>)


我覺得完整的排列組合應該有六組才對，如下：

In [ ]:
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
c = [c for c in combinations([1, 2, 3, 4], 2)]
%f c -->  # 大鍋炒 ;-D 

Next is to apply the second GA variant, mutation, to the results of the crossover stored in the offspring variable using the mutation function inside the GA module. Such function accepts the crossover offspring and returns them after applying uniform mutation. That function is implemented as follows:

In [ ]:
debug = False  # _debug_
def mutation(offspring_crossover):  # 前面洗過牌的子代 shape (4,6)
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):  # 4 個子代一一加以突變
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)  # 一個 random 數
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value  # 驚！突變只發生在某個基因 (#4)上。
        if debug : peforth.push(locals()).ok("bp44>",cmd='to _locals_')     
    return offspring_crossover

It loops through each offspring and adds a uniformly generated random number in the range from -1 to 1 according to this line:

    random_value = numpy.random.uniform(-1.0, 1.0, 1)

Such random number is then added to the gene with index 4 of the offspring according to this line:

    offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    
Note that the index could be changed to any other index. The offspring after applying mutation are as follows:

    [[-0.63698911 -2.8638447   2.93392615 -0.72163167  1.66083721  0.00677938]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.94513681  2.29682254]
     [ 1.96561297  0.51030292  0.52852716  3.78571392  0.45337472  3.5170347 ]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -1.5781162   0.30567304]]

In [35]:
offspring = mutation(offspring_crossover)
%f offspring . cr

[[-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -2.89300931]
 [-1.74953969 -3.56136451 -0.22888881  0.06572773 -2.74471276 -4.01220049]
 [-1.74953969 -3.56136451 -0.22888881 -0.51293995 -3.15336626 -1.44153594]
 [-0.34610757 -0.48601818 -0.21074154 -0.51293995 -3.15336626 -1.70830617]]


Such results are added to the variable offspring_crossover and got returned by the function.

At this point, we successfully produced 4 offspring from the 4 selected parents and we are ready to create the new population of the next generation.

Note that GA is a random-based optimization technique. It tries to enhance the current solutions by applying some random changes to them. Because such changes are random, we are not sure that they will produce better solutions. For such reason, it is preferred to keep the previous best solutions (parents) in the new population. In the worst case when all the new offspring are worse than such parents, we will continue using such parents. As a result, we guarantee that the new generation will at least preserve the previous good results and will not go worse. The new population will have its first 4 solutions from the previous parents. The last 4 solutions come from the offspring created after applying crossover and mutation:

    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

By calculating the fitness of all solutions (parents and offspring) of the first generation, their fitness is as follows:

    [ 18.24112489  17.0688537   15.99527402  14.40299221  -8.46075629  31.73289712   6.10307563  24.08733441]

The highest fitness previously was **18.24112489** but now it is **31.7328971158**. That means that the random changes moved towards a better solution. This is GREAT. But such results could be enhanced by going through more generations. Below are the results of each step for another 4 generations:

    Generation :  1

    Fitness values:

    [ 18.24112489  17.0688537   15.99527402  14.40299221  -8.46075629  31.73289712   6.10307563  24.08733441]

    Selected parents:

    [[ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.94513681  2.29682254]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -1.5781162   0.30567304]
     [-0.63698911 -2.8638447   2.93392615 -1.40103767 -1.20313655  0.30567304]
     [ 3.00912373 -2.745417    3.27131287 -0.72163167  0.7516408   0.00677938]]

    Crossover result:

    [[ 3.00912373 -2.745417    3.27131287 -1.40103767 -1.5781162   0.30567304]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -1.20313655  0.30567304]
     [-0.63698911 -2.8638447   2.93392615 -0.72163167  0.7516408   0.00677938]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.94513681  2.29682254]]

    Mutation result:

    [[ 3.00912373 -2.745417    3.27131287 -1.40103767 -1.2392086   0.30567304]
     [ 2.12480298  2.97122243  3.60375452 -1.40103767 -0.38610586  0.30567304]
     [-0.63698911 -2.8638447   2.93392615 -0.72163167  1.33639943  0.00677938]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -1.13941727  2.29682254]]

    Best result after generation 1 :  34.1663669207
. . . snip . . .
    Generation :  4

    Fitness values

    [ 34.59304326  34.16636692  33.7449326   31.73289712  44.81692352

      33.35989464  36.46723397  37.19003273]

    Selected parents:

    [[ 3.00912373 -2.745417    3.27131287 -1.40103767 -2.20744102  0.30567304]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.44124005  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.37553107  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.20515009  2.29682254]]

    Crossover result:

    [[ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.37553107  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.20515009  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.40103767 -2.20744102  0.30567304]]

    Mutation result:

    [[ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.13382082  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.98105233  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.56909315 -2.27638584  2.29682254]
     [ 3.00912373 -2.745417    3.27131287 -1.40103767 -1.70558545  0.30567304]]

    Best result after generation 4:  44.8169235189


After the above 5 generations, the best result now has a fitness value equal to 44.8169235189 compared to the best result after the first generation which is 18.24112489.

The best solution has the following weights:

    [3.00912373 -2.745417    3.27131287 -1.40103767 -2.20744102  0.30567304]


In [ ]:
# Experimental giving different gene index, the result is similar 

def mutation(offspring_crossover, index):  # 前面洗過牌的子代 shape (4,6)
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):  # 4 個子代一一加以突變
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)  # 一個 random 數
        offspring_crossover[idx, index] = offspring_crossover[idx, index] + random_value  # 驚！突變只發生在某個基因 (index)上。
    return offspring_crossover

In [37]:
import numpy
import GA

#
# The y=target is to maximize this equation ASAP:
#    y = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
#    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
#    What are the best values for the 6 weights w1 to w6?
#    We are going to use the genetic algorithm for the best possible values after a number of generations.
#

# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = len(equation_inputs)

#
# Genetic algorithm parameters:
#     Mating pool size
#     Population size
#

sol_per_pop = 8
num_parents_mating = 4

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

%f new_population ==> # original 

num_generations = 5

for generation in range(num_generations):
    print("Generation : ", generation)

    # Measing the fitness of each chromosome in the population.
    fitness = GA.cal_pop_fitness(equation_inputs, new_population)

    # Selecting the best parents in the population for mating.
    parents = GA.select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = GA.crossover(parents, offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Adding some variations to the offsrping using mutation.
    # offspring_mutation = GA.mutation(offspring_crossover)
    offspring_mutation = mutation(offspring_crossover, 1)  # 改用上面的實驗版

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
    print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))

%f cr 
%f offspring_mutation ==> # after all

# Getting the best solution after iterating finishing all generations.
# At first, the fitness is calculated for each solution in the final generation.
fitness = GA.cal_pop_fitness(equation_inputs, new_population)

# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])


new_population ==>
 [[ 1.96827509  2.04341469  2.18141111  1.21290069  2.66431037  2.65924713]
 [ 0.28903627 -1.00720767 -2.18243685 -1.11505618 -3.58946822 -0.13887248]
 [-0.85422217  2.26897987  2.89642152  2.19151511 -2.13300249  1.02256388]
 [ 0.1840384  -1.05159694 -1.24792706  1.30606635  0.73341364  1.00730912]
 [-1.46312966 -2.64284936  2.58174363 -0.44863616  3.86220586 -3.14776126]
 [ 0.56826096 -0.77048885 -2.6066013   3.60011285  0.58856147 -0.78272734]
 [ 1.00016844  3.68890169 -1.55910163 -3.30176257  0.94530628 -1.61012287]
 [-3.74550804 -2.91499567 -0.90587322  2.10201257  3.04464826  2.79962532]] (<class 'numpy.ndarray'>)
original

Generation :  0
Best result :  39.76344053059758
Generation :  1
Best result :  39.76344053059758
Generation :  2
Best result :  46.32812817024072
Generation :  3
Best result :  46.32812817024072
Generation :  4
Best result :  47.9862437364507

offspring_mutation ==>
 [[-0.85422217  2.26897987  2.89642152 -1.11505618 -3.58946822 -2.53079731]

# --- The End ---